In [10]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/u

In [11]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-28 04:20:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-28 04:20:16 (6.55 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [16]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://tanianguyenavocado-bucket.s3.amazonaws.com/avocado-updated-2020.csv"
spark.sparkContext.addFile(url)
avocado_data_df = spark.read.csv(SparkFiles.get("avocado-updated-2020.csv"), sep=",", header=True, inferSchema=True)

In [17]:
# Show DataFrame
avocado_data_df.show()

+--------+-------------+------------+----------+----------+---------+----------+----------+----------+-----------+------------+----+--------------------+
|    date|average_price|total_volume|      4046|      4225|     4770|total_bags|small_bags|large_bags|xlarge_bags|        type|year|           geography|
+--------+-------------+------------+----------+----------+---------+----------+----------+----------+-----------+------------+----+--------------------+
|1/4/2015|         1.22|    40873.28|    2819.5|  28287.42|     49.9|   9716.46|   9186.93|    529.53|        0.0|conventional|2015|              Albany|
|1/4/2015|         1.79|     1373.95|     57.42|    153.88|      0.0|   1162.65|   1162.65|       0.0|        0.0|     organic|2015|              Albany|
|1/4/2015|          1.0|   435021.49| 364302.39|  23821.16|    82.15|  46815.79|  16707.15|  30108.64|        0.0|conventional|2015|             Atlanta|
|1/4/2015|         1.76|     3846.69|   1500.15|    938.35|      0.0|   1408

In [64]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://tanianguyenavocado-bucket.s3.amazonaws.com/avocado_table.csv"
spark.sparkContext.addFile(url)
avocado_df = spark.read.csv(SparkFiles.get("avocado_table.csv"), sep=",", header=True, inferSchema=True)

In [65]:
# Show DataFrame
avocado_df.show()

+------------+-------------+----------+----+------------+
|        type|average_price|   on_date|year|total_volume|
+------------+-------------+----------+----+------------+
|conventional|         1.22|2015-01-04|2015|    40873.28|
|     organic|         1.79|2015-01-04|2015|     1373.95|
|conventional|          1.0|2015-01-04|2015|   435021.49|
|     organic|         1.76|2015-01-04|2015|     3846.69|
|conventional|         1.08|2015-01-04|2015|   788025.06|
|     organic|         1.29|2015-01-04|2015|    19137.28|
|conventional|         1.01|2015-01-04|2015|    80034.32|
|     organic|         1.64|2015-01-04|2015|     1505.12|
|conventional|         1.02|2015-01-04|2015|    491738.0|
|     organic|         1.83|2015-01-04|2015|     2192.13|
|conventional|          1.4|2015-01-04|2015|   116253.44|
|     organic|         1.73|2015-01-04|2015|      379.82|
|conventional|         0.93|2015-01-04|2015|   5777334.9|
|     organic|         1.24|2015-01-04|2015|   142349.77|
|conventional|

In [29]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://tanianguyenavocado-bucket.s3.amazonaws.com/plu_table.csv"
spark.sparkContext.addFile(url)
plu_data_df = spark.read.csv(SparkFiles.get("plu_table.csv"), sep=",", header=True, inferSchema=True)

In [30]:
# Show DataFrame
plu_data_df.show()

+-------------+----------+----------+---------+--------------------+
|average_price|  plu_4046|  plu_4225| plu_4770|           geography|
+-------------+----------+----------+---------+--------------------+
|         1.22|    2819.5|  28287.42|     49.9|              Albany|
|         1.79|     57.42|    153.88|      0.0|              Albany|
|          1.0| 364302.39|  23821.16|    82.15|             Atlanta|
|         1.76|   1500.15|    938.35|      0.0|             Atlanta|
|         1.08|  53987.31| 552906.04| 39995.03|Baltimore/Washington|
|         1.29|   8040.64|   6557.47|   657.48|Baltimore/Washington|
|         1.01|  44562.12|  24964.23|  2752.35|               Boise|
|         1.64|      1.27|    1129.5|      0.0|               Boise|
|         1.02|   7193.87| 396752.18|   128.82|              Boston|
|         1.83|      8.66|    939.43|      0.0|              Boston|
|          1.4|   3267.97|  55693.04|   109.55|   Buffalo/Rochester|
|         1.73|       0.0|     59.

In [31]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://tanianguyenavocado-bucket.s3.amazonaws.com/size_table.csv"
spark.sparkContext.addFile(url)
size_data_df = spark.read.csv(SparkFiles.get("size_table.csv"), sep=",", header=True, inferSchema=True)

In [32]:
# Show DataFrame
size_data_df.show()

+------------+----------+----------+-----------+----------+
|        type|small_bags|large_bags|xlarge_bags|total_bags|
+------------+----------+----------+-----------+----------+
|conventional|   9186.93|    529.53|        0.0|   9716.46|
|     organic|   1162.65|       0.0|        0.0|   1162.65|
|conventional|  16707.15|  30108.64|        0.0|  46815.79|
|     organic|   1071.35|    336.84|        0.0|   1408.19|
|conventional| 137146.07|   3990.61|        0.0| 141136.68|
|     organic|   3881.69|       0.0|        0.0|   3881.69|
|conventional|    6064.3|   1691.32|        0.0|   7755.62|
|     organic|    186.67|    187.68|        0.0|    374.35|
|conventional|  87406.84|    256.29|        0.0|  87663.13|
|     organic|   1244.04|       0.0|        0.0|   1244.04|
|conventional|  57182.88|       0.0|        0.0|  57182.88|
|     organic|     320.0|       0.0|        0.0|     320.0|
|conventional| 477193.38|  47882.56|     3375.8| 528451.74|
|     organic|   9144.15|       0.0|    

In [68]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-finalavocado.ce3rvilzockn.us-east-2.rds.amazonaws.com:5432/avocado_final"
config = {"user":"postgres",
          "password": "ngoc2mna12",
          "driver":"org.postgresql.Driver"}

In [61]:
# Write DataFrame to active_user table in RDS
plu_data_df.write.jdbc(url=jdbc_url, table='plu', mode=mode, properties=config)

In [62]:
# Write DataFrame to active_user table in RDS
size_data_df.write.jdbc(url=jdbc_url, table='size', mode=mode, properties=config)

In [69]:
# Write DataFrame to active_user table in RDS
avocado_df.write.jdbc(url=jdbc_url, table='avocado_table', mode=mode, properties=config)